In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# read in csv 
df = pd.read_csv('data/df_final_2023_08_25')
# shape 
print("Shape: ", df.shape)

Shape:  (907485, 16)


In [2]:
print("Set the printing size of the pandas df: ")
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Set the printing size of the pandas df: 


In [3]:
print("Deleting the discontious SCC where period_index==-1")
print("Shape: ", df.shape)
df = df[df["period_index"] != -1]
print("Shape: ", df.shape)

Deleting the discontious SCC where period_index==-1
Shape:  (907485, 16)
Shape:  (671868, 16)


In [4]:
print("Shift scc to the first column")
# shift column 'scc' to first position
first_column = df.pop('scc')
# insert column using insert(position,column_name, first_column) function
df.insert(0, 'scc', first_column)

Shift scc to the first column


In [5]:
print("For convinent, make milkng_date the string")
milkng_date_only = df['milkng_date'].apply(lambda x: str(x))
df['milkng_date_only'] = milkng_date_only

For convinent, make milkng_date the string


# Pre-processing (de-trending, normalize cow, dataset chunking)

In [6]:
# ingore future warning
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
# surprass chain warning
pd.set_option('mode.chained_assignment', None)
#warnings.filterwarnings("ignore", message=r"Passing", category=SettingWithCopyWarning)

In [7]:
print("All the id of the herds: ")
print(len(df["hrd_ida"].unique().tolist()), df["hrd_ida"].unique().tolist())

All the id of the herds: 
40 [4, 50, 29, 34, 47, 9, 45, 31, 2, 30, 15, 19, 23, 42, 5, 7, 38, 40, 20, 36, 21, 39, 1, 8, 10, 18, 32, 25, 3, 22, 11, 41, 27, 6, 0, 14, 49, 37, 28, 26]


In [8]:
print("Rescale and bucket the scc value")

def conditions(i): 
    if i <= 50: return 50
    if i > 50 and i <= 100: return i
    if i > 100 and i <= 200: return i 
    if i > 200 and i <= 500: return i + 75
    
    if i > 500 and i <= 1000: return i + 125
    if i > 1000 and i <= 1500: return i + 250
    if i > 1500 and i <= 2000: return i + 500
    if i > 2000: return 3000



Rescale and bucket the scc value


### De-trending

### Chunking for time-series model

In [128]:
# 1. Given a dataset of a cow, find the start and end index of valid period (continous dim and the same period index) 

# 2. Given a valid period, chunk to X and y 

# 3. Use an overall function to loop through cows in a herd

In [129]:
# 1. Given a dataset of a cow, find the start and end index of valid period (continous dim and the same period index) 
""" 
func: 
    to find the start and end index of valid period 
args: 
    ls_dim: list[float] 
    ls_period_index: list[int] 
return: 
    ls_index: a list of start and end index of this cow: list[int]
"""
def find_valid_perod(ls_dim, ls_period_index): 
    
    # check the length of ls_dim 
    if len(ls_dim) <= 2: 
        return [0, len(ls_dim)]
        
    # init result and append the start index
    ls_index = [0]  
    
    # loop through the two input lists 
    for i in range(len(ls_dim)-1): 
        # if dim jumped or period index changed 
        if ((ls_dim[i+1] - 1) != ls_dim[i]) or (ls_period_index[i+1] != ls_period_index[i]): 
            # append new end index 
            ls_index.append(i) 
            
    # check for the last element  
    if (ls_dim[-1]-1 == ls_dim[-2]) and (ls_period_index[-1] == ls_period_index[-2]): 
        # append the last 
        ls_index.append(len(ls_dim))
        
    return ls_index
    

In [130]:
# 2. Given a valid period, chunk to X and y 
""" 
func: 
    given a valid period, create 
    dataset for seq2seq models X, y 
args: 
    valid_period: dataframe (the target (scc_std) has index 0!)
    X_len: int (length of input time period)
    y_len: int (length of output time period)
return: 
    don't return, keep appending ls_X, ls_y
"""
def chunk_to_X_y(valid_period, X_len, y_len, ls_X, ls_y):
    
    # check length 
    if valid_period.shape[0] < X_len + y_len: 
        return None

    # loop by the rows (start, end, step)
    for i in range(0, valid_period.shape[0]-X_len-y_len, y_len): 
        # append to X 
        ls_X.append(valid_period.iloc[i:i+X_len, :].to_numpy())
        # append to y 
        ls_y.append(valid_period.iloc[i+X_len:i+X_len+y_len, 0].to_numpy())
                    

In [131]:
# 3. Use an overall function to loop through cows in a herd
""" 
func: 
    loop through cows and chunking 
args: 
    df_stded: dataframe
    ls_predictors: list of variables keeped in prediction (list[str])
    X_len, y_len: the length of X and y (predictors and predicitons) (int, int)
return: 

"""
def chunk_herd(df_stded, ls_predictors):
    
    # list of cow id 
    ls_cow_id = df_stded['anm_ida'].unique().tolist() 

    # store valid_period 
    ls_df_period = []
    
    # looping cow id 
    for idx in ls_cow_id:
        
        # obtain cow data by index
        df_cow = df_stded[df_stded['anm_ida']==idx]

        """using func: """
        ls_index = find_valid_perod(df_cow['dim'].tolist(), df_cow['period_index'].tolist())

        # loop through ls_index for each valid period 
        for idx in range(len(ls_index)-1):
        
            # obtain a period from ls_index 
            valid_period = df_cow.iloc[ls_index[idx]:ls_index[idx+1], :]

            # keep only the predictor variables
            valid_period = valid_period[ls_predictors]
        
            # append to ls_df_period
            ls_df_period.append(valid_period)
            
    return ls_df_period

In [132]:
""" 
func: 
    to process a herd of data 
arg: 
    df_4: pd.Dataframe
    var_list_dim: list to correct for dim (list[str])
    var_list_date: list to correct for date (list[str])
    X_len: int
    y_len: int
    ls_predictors: list of vars keep (list[str])
    one_herd=True: boolean 
return: 
    ls_X: numpy 3d
    ls_y: numpy 2d 
    scc_mean: int
    scc_var: int
"""
def process_data_per_herd(df, var_list_dim, var_list_date, X_len, y_len, ls_predictors, one_herd=True):

    if one_herd:
        print("Put scc values in buckets to regulize regression model")
        """using func: """
        df['scc_bucket']=df['scc'].apply(conditions)
        print("check nan: ", df['scc_bucket'].isnull().sum())

    print("Correcting for dim...")
    for var_name in var_list_dim:
        # obtain the list of mean
        mean = df.groupby('dim').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['dim']], axis=1)

    print("Correcting for milkng_date_only...")
    for var_name in var_list_date:
        # obtain the list of mean
        mean = df.groupby('milkng_date_only').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['milkng_date_only']], axis=1)

    # we can pick if to standardize from here or not 
    if one_herd:
        print("Standardize SCC and storing")
        scc_var = df['scc_bucket'].std() 
        scc_mean = df['scc_bucket'].mean() 
        df["scc"] = (df['scc_bucket'] - scc_mean) / scc_var
        print("The mean and variance: ", scc_mean, scc_var)
    else: 
        scc_mean, scc_var = -1, -1
        print("Not standardizing scc per herd: ", scc_mean, scc_var)

    # standardization: except for scc where we have already stded
    for i in range(1, len(ls_predictors)): 
        df[ls_predictors[i]] = (df[ls_predictors[i]] - df[ls_predictors[i]].mean()) / df[ls_predictors[i]].std()

    """using func: """
    ls_X, ls_y = chunk_herd(df,  ls_predictors)
    
    return ls_X, ls_y, scc_mean, scc_var

# Sequence to sequence regression for each period

In [133]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm_notebook
from itertools import product
from typing import Union

import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [134]:
def ARIMAX_model(y_train, y_test, pdq_tuple_ls, best_order, tune):
    
    # if ture 
    if tune:
        # init 
        results = [] 
        # looping
        for pdq_tuple in pdq_tuple_ls:
            # define model
            model = SARIMAX(
                y_train,
                order=pdq_tuple).fit(disp=False)

            # obtain AIC value
            aic = model.aic
            # append to the result
            results.append([pdq_tuple, aic])

        # obtain the order 
        results = np.array(results)
        idx = np.argmin(results[:, 1])
        best_order = results[idx, 0] 

    # define model
    model = SARIMAX(
        y_train,
        order=best_order).fit(disp=False)
        
    # get predictions
    predictions = model.get_prediction(exog=y_test)
    y_pred = predictions.predicted_mean
    
    return y_pred, best_order

In [135]:
""" 
func: 
    to evaluate regression by 
    converting to classification problem 
args: 
    y_test_value: numpy (number of period, length)
    y_pred_value: numpy (number of period, length)
    scc_threshold: the scc threshold set to pick out mastitis in the predicion set 
    buffer = 5: int (the delay of mastitis occurence)
return: 
    SP, SE, AUC
"""
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
def regress_to_classify(test, pred, scc_threshold, buffer = 5): 
    
    # set result y_pred as the num of rows (example)
    y_pred = [0 for i in range(pred.shape[0])]
    y_test = [0 for i in range(pred.shape[0])]

    # loop rows 
    for i in range(pred.shape[0]): 
        one_pred = pred[i, -buffer:]
        # set 1 in y_pred base on the threshold 
        if max(one_pred) >= scc_threshold: y_pred[i] = 1
        one_test = test[i, -buffer:]
        # set 1 in y_test base on the threshold: here is 2500 because of bucketing method make 2000 2500
        if max(one_test) >= 2500: y_test[i] = 1
    
    """using sklearn"""
    try:
        # this func requie y_test to have at least two classes
        auc = roc_auc_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sp = tn / (tn + fp)
        se = tp / (tp + fn)
    except ValueError:
        print("Invalid threshold")
        sp, se, auc = 0.1, 0.1, 0.1

    print("sp, se, auc: ", sp, se, auc)
    return sp, se, auc

# Combine data-processing and prediction for herds

### Training all the herds together

In [137]:
""" 
func: 
    the final function from dataframe to X, y 
args: 
    df: pd.Dataframe (raw df) 
    var_list_dim: list[str] 
    var_list_date: list[str] 
    X_len: int (input period) 
    y_len: int (output period) 
    ls_predictors: list[str]
return: 
    ls_X_ls: np.array (examples, input period, variables)
    ls_y_ls: np.array (examples, input period)
    scc_mean, scc_var: float 
"""
def process_all_herd(df, var_list_dim, var_list_date, ls_predictors):

    """using func: conditions"""
    df['scc']=df['scc'].apply(conditions)
    print("check nan: ", df['scc'].isnull().sum())

    print("Correcting for dim...")
    for var_name in var_list_dim:
        # obtain the list of mean
        mean = df.groupby('dim').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['dim']], axis=1)

    print("Correcting for milkng_date_only...")
    for var_name in var_list_date:
        # obtain the list of mean
        mean = df.groupby('milkng_date_only').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['milkng_date_only']], axis=1)

    # we can pick if to standardize from here or not
    scc_var = df['scc'].std() 
    scc_mean = df['scc'].mean() 

    # standardization:
    for i in range(len(ls_predictors)): 
        df[ls_predictors[i]] = (df[ls_predictors[i]] - df[ls_predictors[i]].mean()) / df[ls_predictors[i]].std()

    """using func: """
    ls_df_valid = chunk_herd(df,  ls_predictors)

    return ls_df_valid, scc_mean, scc_var

In [155]:
""" 
func: 
    define model, split train&test, and evaluate
args: 
    ls_X: numpy 3d
    ls_y: numpy 2d
    scc_mean: int
    scc_var: int
    EPOCHS: int
return: 
    y_pred_value, y_test_value: numpy 2d
    SP, SE, AUC: int
"""
def train_and_evaluate_herd(ls_X, ls_y, X_len, y_len, scc_mean, scc_var, len_ls_predictors, pdq_tuple_ls):

    # split into training and testing
    training_ratio = 0.8
    # split and convert to numpy array format (required by tf)
    train_set = (np.array(ls_X[:int(training_ratio*len(ls_X))]), np.array(ls_y[:int(training_ratio*len(ls_y))]))
    test_set = (np.array(ls_X[int(training_ratio*len(ls_X)):]), np.array(ls_y[int(training_ratio*len(ls_y)):]))

    print("Check length of X and y: ", len(ls_X), len(ls_y))
    print("Check length of train and test: ", len(train_set[0]), len(test_set[0]))
    print("training_ratio: ", training_ratio)

    """using func: """
    # obtain X_train, y_train as numpy array
    X_train = train_set[0] 
    y_train = train_set[1] 
    X_test = test_set[0] 
    y_test = test_set[1] 
    
    # init y_pred 
    y_pred = np.zeros(y_test.shape)
    
    print("y_pred: ", y_pred.shape)
    print("y_test: ", y_test.shape, np.isnan(np.min(y_test)))
    print("total: ", X_test.shape[0])
    # init 
    tuning = True
    best_order = (0, 0, 0)
    # looping to use ARIMA 
    for i in range(X_test.shape[0]-1): 
        if i % 100 == 0:
            print("i, best_order: ", i, best_order)
        if i % 10 == 0: 
            tuning = True
        y_pred_i, best_order = ARIMAX_model(y_test[i, :], y_test[i+1, :], pdq_tuple_ls, best_order, tune=tuning)
        tuning = False
        y_pred[i+1, :] = y_pred_i
        
    print("scc_mean, scc_var: ", scc_mean, scc_var)
    #print("Average error: ", scc_mean * valid_mae)

    print("Making prediction...")
    y_pred_value =  (y_pred * scc_var) + scc_mean
    y_test_value = (y_test * scc_var) + scc_mean
    
    # select scc_threshold based on herds
    threshold_ls = [scc_mean, scc_mean+scc_var, 1000, 2000, 2500]
    #threshold_ls = [scc_mean]
    performance_ls = [0.1, 0.1, 0.1]
    # set init AUC 
    AUC_max = 0.1 
    # looping for threshold 
    for i in threshold_ls:    
        """using func: regress_to_classify"""
        SP, SE, AUC = regress_to_classify(y_test_value, y_pred_value, scc_threshold=i)
        print("SP, SE, AUC: ", SP, SE, AUC)
        # pick by AUC
        if AUC >= AUC_max:
            # append to performance
            performance_ls = [SP, SE, AUC]
            # reset AUC_max 
            AUC_max = AUC
    
    # end looping 
    print()
    
    return y_pred, y_test, performance_ls[0], performance_ls[1], performance_ls[2]

In [138]:
# init list with variables needed to be corrected by dim 
var_list_dim = ["mk_wgt", "min_time", "milk_flow_max", "ft_pcnt"]
# init list with variables needed to be corrected by dim 
var_list_date = ["milkng_temp"]
# a list of predictor variables: 1 target + 6 variables
ls_predictors = ["scc", "mk_wgt", "min_time", "milk_flow_max", "milkng_temp", 'prep_time']

"""using func: """
df_4 = df[df['hrd_ida']==4]
ls_df_valid, scc_mean, scc_var = process_all_herd(df_4, var_list_dim, var_list_date, ls_predictors)

check nan:  0
Correcting for dim...
Correcting for milkng_date_only...


In [140]:
# ls_df_valid[0]

In [141]:
# X length is the length of input X 
X_len = 30
# y length is the length of output y 
y_len = 10

ls_X, ls_y = [], [] 
for i in range(len(ls_df_valid)): 
    valid_period = ls_df_valid[i]
    chunk_to_X_y(valid_period, X_len, y_len, ls_X, ls_y)

In [146]:
# generate 
from itertools import product
p = range(0, 2)
q = range(0, 2)
d = range(0, 2)
parameters = product(p, q, d)
order_list = list(parameters)
print("order_list: ", order_list)

_, _, SP, SE, AUC = train_and_evaluate_herd(ls_X, ls_y, X_len, y_len, scc_mean, scc_var, len(ls_predictors), order_list)

order_list:  [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Check length of X and y:  2555 2555
Check length of train and test:  2044 511
training_ratio:  0.8
y_pred:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_test:  [[ 5.66115054  2.75668841  1.60389696 ...  5.66115054  3.03262253
   2.31723777]
 [ 2.22934764  1.53849036  2.43578724 ...  2.64018289  2.24978721
   2.44396307]
 [ 2.27431468  2.19460038  2.77712797 ...  1.77150139  1.24416063
   3.96671063]
 ...
 [-0.27449903 -0.27449903 -0.21113638 ... -0.21522429  0.83332538
   0.26306152]
 [ 0.40000661  0.23649009  0.35299561 ... -0.29085068  0.15677579
   0.15677579]
 [-0.30720233 -0.16616933  0.09545709 ...  0.09954501  0.09954501
   1.50578705]]
total:  511
i, best_order:  0 (0, 0, 0)
i, best_order:  100 (0, 1, 0)
i, best_order:  200 (0, 1, 1)
i, best_order:  300 (0, 1, 0)
i, 

# Experimenting

In [156]:
ls_df_valid, scc_mean, scc_var = process_all_herd(df, var_list_dim, var_list_date, ls_predictors)

check nan:  0
Correcting for dim...
Correcting for milkng_date_only...


In [157]:
# init result_ls 
result_ls = [] 
# X length is the length of input X, y length is the length of output y 
ls_X_len_y_len = [(21, 5), (28, 5), (35, 5), (21, 8), (28, 8), (35, 8), (21, 10), (28, 10), (35, 10)]

for pair in ls_X_len_y_len:
    X_len, y_len = pair[0], pair[1]
    print("========================================")
    print("X_len, y_len: ", X_len, y_len)
    
    # init X, y
    ls_X, ls_y = [], [] 
    # looping ls_df_valid (list[df])
    for i in range(len(ls_df_valid)): 
        # obtain period
        valid_period = ls_df_valid[i]
        """using func: """
        chunk_to_X_y(valid_period, X_len, y_len, ls_X, ls_y)
        
    # here 
    """using func: train_and_evaluate_herd"""
    y_pred_value, y_test_value, SP, SE, AUC = train_and_evaluate_herd(ls_X, ls_y, X_len, y_len, scc_mean, scc_var, len(ls_predictors), order_list)
    result_ls.append([SP, SE, AUC])

print(result_ls)

X_len, y_len:  21 5
Check length of X and y:  81403 81403
Check length of train and test:  65122 16281
training_ratio:  0.8
y_pred:  (16281, 5)
y_test:  (16281, 5) False
total:  16281
i, best_order:  0 (0, 0, 0)
i, best_order:  100 (0, 1, 0)
i, best_order:  200 (0, 1, 0)
i, best_order:  300 (0, 1, 0)
i, best_order:  400 (0, 1, 0)
i, best_order:  500 (0, 1, 0)
i, best_order:  600 (0, 0, 0)
i, best_order:  700 (0, 1, 0)
i, best_order:  800 (0, 1, 0)
i, best_order:  900 (0, 1, 0)
i, best_order:  1000 (0, 1, 0)
i, best_order:  1100 (0, 1, 0)
i, best_order:  1200 (0, 1, 0)
i, best_order:  1300 (0, 1, 0)
i, best_order:  1400 (0, 1, 0)
i, best_order:  1500 (0, 1, 0)
i, best_order:  1600 (0, 1, 0)
i, best_order:  1700 (0, 1, 0)
i, best_order:  1800 (0, 1, 0)
i, best_order:  1900 (0, 0, 0)
i, best_order:  2000 (0, 1, 0)
i, best_order:  2100 (0, 1, 0)
i, best_order:  2200 (0, 1, 0)
i, best_order:  2300 (0, 1, 1)
i, best_order:  2400 (0, 1, 0)
i, best_order:  2500 (0, 1, 1)
i, best_order:  2600 (

i, best_order:  6200 (0, 1, 0)
i, best_order:  6300 (0, 1, 0)
i, best_order:  6400 (0, 1, 0)
i, best_order:  6500 (0, 1, 0)
i, best_order:  6600 (0, 1, 0)
i, best_order:  6700 (0, 1, 0)
i, best_order:  6800 (0, 1, 0)
i, best_order:  6900 (0, 1, 0)
i, best_order:  7000 (0, 1, 0)
i, best_order:  7100 (0, 1, 0)
i, best_order:  7200 (0, 1, 0)
i, best_order:  7300 (0, 1, 0)
i, best_order:  7400 (0, 0, 0)
i, best_order:  7500 (0, 1, 0)
i, best_order:  7600 (0, 1, 1)
i, best_order:  7700 (0, 1, 0)
i, best_order:  7800 (0, 1, 0)
i, best_order:  7900 (0, 1, 0)
i, best_order:  8000 (0, 0, 0)
i, best_order:  8100 (0, 1, 0)
i, best_order:  8200 (0, 1, 1)
i, best_order:  8300 (0, 1, 0)
i, best_order:  8400 (0, 1, 0)
i, best_order:  8500 (0, 1, 0)
i, best_order:  8600 (0, 1, 0)
i, best_order:  8700 (0, 1, 0)
i, best_order:  8800 (0, 1, 0)
i, best_order:  8900 (0, 1, 0)
i, best_order:  9000 (0, 1, 0)
i, best_order:  9100 (0, 1, 0)
i, best_order:  9200 (0, 1, 0)
i, best_order:  9300 (0, 1, 0)
i, best_

i, best_order:  400 (0, 1, 0)
i, best_order:  500 (0, 1, 0)
i, best_order:  600 (0, 0, 1)
i, best_order:  700 (0, 1, 0)
i, best_order:  800 (0, 1, 0)
i, best_order:  900 (0, 1, 1)
i, best_order:  1000 (0, 0, 1)
i, best_order:  1100 (0, 1, 1)
i, best_order:  1200 (0, 1, 0)
i, best_order:  1300 (0, 1, 0)
i, best_order:  1400 (1, 1, 0)
i, best_order:  1500 (0, 0, 1)
i, best_order:  1600 (0, 1, 0)
i, best_order:  1700 (0, 1, 0)
i, best_order:  1800 (0, 1, 0)
i, best_order:  1900 (0, 1, 0)
i, best_order:  2000 (0, 1, 0)
i, best_order:  2100 (0, 0, 0)
i, best_order:  2200 (1, 1, 0)
i, best_order:  2300 (0, 1, 0)
i, best_order:  2400 (0, 1, 1)
i, best_order:  2500 (0, 1, 0)
i, best_order:  2600 (0, 1, 1)
i, best_order:  2700 (0, 0, 0)
i, best_order:  2800 (0, 1, 0)
i, best_order:  2900 (0, 1, 1)
i, best_order:  3000 (0, 0, 0)
i, best_order:  3100 (1, 1, 0)
i, best_order:  3200 (0, 1, 1)
i, best_order:  3300 (0, 0, 0)
i, best_order:  3400 (0, 1, 0)
i, best_order:  3500 (0, 1, 1)
i, best_order:

i, best_order:  1800 (0, 1, 0)
i, best_order:  1900 (0, 0, 0)
i, best_order:  2000 (0, 1, 0)
i, best_order:  2100 (0, 1, 0)
i, best_order:  2200 (0, 1, 1)
i, best_order:  2300 (1, 1, 0)
i, best_order:  2400 (0, 1, 0)
i, best_order:  2500 (0, 1, 0)
i, best_order:  2600 (0, 1, 0)
i, best_order:  2700 (0, 0, 0)
i, best_order:  2800 (0, 0, 0)
i, best_order:  2900 (0, 1, 0)
i, best_order:  3000 (0, 1, 0)
i, best_order:  3100 (0, 1, 0)
i, best_order:  3200 (0, 1, 0)
i, best_order:  3300 (0, 1, 0)
i, best_order:  3400 (0, 1, 0)
i, best_order:  3500 (0, 1, 1)
i, best_order:  3600 (0, 1, 0)
i, best_order:  3700 (0, 1, 0)
i, best_order:  3800 (0, 1, 0)
i, best_order:  3900 (0, 1, 0)
i, best_order:  4000 (0, 1, 0)
i, best_order:  4100 (0, 0, 0)
i, best_order:  4200 (0, 1, 0)
i, best_order:  4300 (1, 1, 0)
i, best_order:  4400 (0, 1, 0)
i, best_order:  4500 (0, 0, 0)
i, best_order:  4600 (0, 1, 0)
i, best_order:  4700 (0, 0, 0)
i, best_order:  4800 (0, 0, 0)
i, best_order:  4900 (0, 1, 0)
i, best_

i, best_order:  6600 (0, 1, 1)
scc_mean, scc_var:  256.84740454970324 537.2763125617959
Making prediction...
sp, se, auc:  0.0 1.0 0.5
SP, SE, AUC:  0.0 1.0 0.5
sp, se, auc:  0.6202409638554217 0.8274111675126904 0.723826065684056
SP, SE, AUC:  0.6202409638554217 0.8274111675126904 0.723826065684056
sp, se, auc:  0.8480321285140562 0.6522842639593909 0.7501581962367234
SP, SE, AUC:  0.8480321285140562 0.6522842639593909 0.7501581962367234
sp, se, auc:  0.9137349397590362 0.5558375634517766 0.7347862516054064
SP, SE, AUC:  0.9137349397590362 0.5558375634517766 0.7347862516054064
sp, se, auc:  0.9635341365461847 0.434010152284264 0.6987721444152244
SP, SE, AUC:  0.9635341365461847 0.434010152284264 0.6987721444152244
sp, se, auc:  0.9778313253012049 0.35279187817258884 0.6653116017368967
SP, SE, AUC:  0.9778313253012049 0.35279187817258884 0.6653116017368967

X_len, y_len:  35 10
Check length of X and y:  27949 27949
Check length of train and test:  22359 5590
training_ratio:  0.8
y_pred

In [1]:
result_ls = [[0.855276907001045, 0.7141382868937048, 0.7847075969473749], [0.8540029000992139, 0.7376058041112454, 0.7958043521052297], [0.8564794141287845, 0.710334788937409, 0.7834071015330967], [0.8530678148546824, 0.6626297577854672, 0.7578487863200748], [0.9156032849020846, 0.5946502057613169, 0.7551267453317008], [0.909482119008755, 0.6070588235294118, 0.7582704712690833], [0.8560144124168514, 0.6252723311546841, 0.7406433717857677], [0.8480321285140562, 0.6522842639593909, 0.7501581962367234], [0.9163981042654028, 0.6, 0.7581990521327014]]

### Plot the results 

In [10]:
""" 
func: 
    to write the label on plots 
args: 
    dates, temp: list of x and y in the plot 
    plt: the plot object
return: 
    annotated plot
"""
import matplotlib.pyplot as plt
import numpy as np
def write_label(dates, temp, plt): 
    for x, y in zip(dates, temp):
        label = y
        plt.annotate(label, (x, y),
                     xycoords="data",
                     textcoords="offset points",
                     xytext=(0, 10), ha="center")

""" 
func: 
    to plot for a model's result 
args: 
    log_ls: list[list[float]] the results of this model
    model_name: str
    path: str
return: 
    name_ls: list[str]
"""
def plot_one_model(log_ls, model_name, path, char_ls):
    
    from matplotlib.pyplot import figure

    # init methods
    method_str = [' 21 days', ' 28 days', '35 days']
    # init method id (0, 1, 2)
    idx_method_ls = [0, 1, 2]
    # total num of methods
    shrink_method = 3 
    # init name 
    name_ls = []

    # looping for idx_method 
    for idx_method in idx_method_ls: 

        # init list for SP, SE, AUC 
        SP, SE, AUC = [], [], [] 

        # loop the list 
        for i in range(idx_method, len(log_ls), shrink_method):
            # specificity
            sp = round(log_ls[i][0]*100, 2)
            SP.append(sp)
            # sensitifity
            se = round(log_ls[i][1]*100, 2)
            SE.append(se)
            # auc
            auc = round(log_ls[i][2]*100, 2)
            AUC.append(auc)   

        # reserse the list
        SP, SE, AUC = SP[::-1], SE[::-1], AUC[::-1]
        x_axis = ['5 days', '3 days', '1 days']

        plt.plot(x_axis, SP, label = 'specificity', color="indigo", marker = 'o')
        """using func: """
        write_label(x_axis, SP, plt)
        plt.plot(x_axis, SE, label = 'sensitivity', color="indigo", marker = 's')
        """using func: """
        write_label(x_axis, SE, plt)
        plt.plot(x_axis, AUC, label = 'AUC', color="forestgreen", marker = '^')
        """using func: """
        write_label(x_axis, AUC, plt)
        plt.legend() 
        plt.title(model_name + " with input length: " + method_str[idx_method])
        plt.xlabel("(" + char_ls[idx_method] + ")")
        
        # saving 
        name = model_name + "_" + method_str[idx_method]
        name_ls.append(name)
        plt.gcf().set_size_inches(5, 5)
        plt.savefig(path+name, dpi=200)
        plt.close()
        
    return name_ls



In [11]:
result_ls_arima = [[0.855276907001045, 0.7141382868937048, 0.7847075969473749], [0.8540029000992139, 0.7376058041112454, 0.7958043521052297], [0.8564794141287845, 0.710334788937409, 0.7834071015330967], [0.8530678148546824, 0.6626297577854672, 0.7578487863200748], [0.9156032849020846, 0.5946502057613169, 0.7551267453317008], [0.909482119008755, 0.6070588235294118, 0.7582704712690833], [0.8560144124168514, 0.6252723311546841, 0.7406433717857677], [0.8480321285140562, 0.6522842639593909, 0.7501581962367234], [0.9163981042654028, 0.6, 0.7581990521327014]]
name_arima = plot_one_model(result_ls_arima, 'ARIMA', 'plot/Results_regression_new/', 'adg')

result_ls_lstm = [[0.8500522466039707, 0.8307533539731682, 0.8404028002885694], [0.8536213080973822, 0.841596130592503, 0.8476087193449425], [0.8527284093953738, 0.851528384279476, 0.8521283968374248], [0.8526372443487621, 0.801038062283737, 0.8268376533162496], [0.8577384712571068, 0.8106995884773662, 0.8342190298672365], [0.8576940198842559, 0.8117647058823529, 0.8347293628833045], [0.8514412416851441, 0.7930283224400871, 0.8222347820626157], [0.8393574297188755, 0.8121827411167513, 0.8257700854178134], [0.8490995260663508, 0.8380952380952381, 0.8435973820807945]]
name_lstm = plot_one_model(result_ls_lstm, 'LSTM', 'plot/Results_regression_new/', 'beh')

result_ls_transformer = result_ls = [[0.8738897596656218, 0.8121775025799793, 0.8430336311228006], [0.8769747386094787, 0.8331318016928658, 0.8550532701511722], [0.8723765294275252, 0.834061135371179, 0.853218832399352], [0.8698600645855759, 0.7906574394463668, 0.8302587520159713], [0.8852811118130133, 0.7860082304526749, 0.8356446711328441], [0.8904882029974773, 0.7929411764705883, 0.8417146897340327], [0.8812361419068736, 0.7734204793028322, 0.8273283106048529], [0.8677911646586345, 0.7791878172588832, 0.8234894909587588], [0.865781990521327, 0.8444444444444444, 0.8551132174828857]]
name_transformer = plot_one_model(result_ls_transformer, 'TF', 'plot/Results_regression_new/', 'cfi')

In [12]:
""" 
func: 
    concate plots 
args: 
    name_ls_log, name_ls_tree, name_ls_mlp: list[str]
return: 
    a plot
"""
def concat_plots(name_ls, path, name):
    
    temp = name_ls
    figa = plt.imread(path + temp[0] + '.png')
    figb = plt.imread(path + temp[1] + '.png')
    figc = plt.imread(path + temp[2] + '.png')
    figd = np.concatenate((figa,figb), axis=0)
    fige = np.concatenate((figd,figc), axis=0) 
                   
    fname = path + name + ".png"
    plt.imsave(fname, fige)

In [13]:

concat_plots(name_arima, 'plot/Results_regression_new/', 'ARIMA')

concat_plots(name_lstm, 'plot/Results_regression_new/', 'LSTM')

concat_plots(name_transformer, 'plot/Results_regression_new/', 'TF')

# Additional codes

In [14]:
print("concat three models: ")
path = 'plot/Results_regression_new/'

figa = plt.imread(path + 'ARIMA' + '.png')
figb = plt.imread(path + 'LSTM' + '.png')
figc = plt.imread(path + 'TF' + '.png')
figd = np.concatenate((figa,figb), axis=1)
fige = np.concatenate((figd,figc), axis=1) 

fname = path + "Seq2Seq_Regression_new" + ".png"
plt.imsave(fname, fige)

concat three models: 
